In [5]:
import pandas as pd
from openai import OpenAI
import os

#asd
#asd
i = 1

os.environ["OPENAI_API_KEY"] = #ADD API Key


client = OpenAI()


instructions="You are a fridge inventory bot. Use the provided functions to answer questions.",
model="gpt-4-turbo",


file = client.files.create(
  file=open("./data/fridge_inventory.csv", "rb"),
  purpose='assistants'
)

tools=[
    {
      "type": "function",
      "function": {
        "name": "edit_inventory_table",
        "description": "Edit the inventory table of the items that are in the fridge. Use this when items are added or removed from the fridge, or whenever information about the quantity or quality of items in the fridge are stated",
        "parameters": {
          "type": "object",
          "properties": {
            "item": {
              "type": "string",
              "description": "The name of the food item that is in or is being added to the inventory of the fridge"
            },
            "count_type": {
              "type": "string",
              "enum": ["add", "remove", "total"],
              "description": "The type of activity that is being done to the food item. Set to --add-- if items are being added to the fridge. Set to -- remove -- if items are being removed. Set to -- total -- if the total number of item type in the fridge is stated "
            },
            "quantity": {
              "type": "integer",
              "description": "The number of items in the user query. E.g. if -- remove 3 milks -- is the query, set it to 3. In a special case, if -- remove all -- or something similar is stated, set to 10000"
            }
            
          },
          "required": ["item", "count_type", "quantity"]
        }
      }
    },
    {"type": "code_interpreter"}
]

In [7]:
def run_conversation(messages,tools):
    # Step 1: send the conversation and available functions to the model
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function

    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "edit_inventory_table": edit_inventory_table,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                item=function_args.get("item"),
                quantity=function_args.get("quantity"),
                count_type = function_args.get("count_type")
            )
    return response_message
#print(run_conversation(messages,tools))

In [8]:
import pandas as pd


def edit_inventory_table(item, count_type, quantity):


    print(f"FUNCTION CALLED: {item}, {count_type}, {quantity}")
    df = pd.read_csv('./data/fridge_inventory.csv')
    
    if count_type == "add":
        if item in df['item'].values:
            df.loc[df['item'] == item, 'quantity'] += quantity
        else:
            df = pd.concat([df, pd.DataFrame([{'item': item, 'quantity': quantity}])], ignore_index=True)
    elif count_type == "remove":
        if item not in df['item'].values:
            raise ValueError(f"No such item '{item}' exists in the inventory.")
        else:
            current_quantity = df.loc[df['item'] == item, 'quantity'].values[0]
            new_quantity = current_quantity - quantity
            if new_quantity < 0:
                raise ValueError(f"Cannot remove {quantity} {item}(s). Inventory has only {current_quantity} {item}(s).")
            df.loc[df['item'] == item, 'quantity'] = new_quantity
    elif count_type == "total":
        if item not in df['item'].values:
            df = pd.concat([df, pd.DataFrame([{'item': item, 'quantity': quantity}])], ignore_index=True)
        else:
            df.loc[df['item'] == item, 'quantity'] = quantity
    else:
        raise ValueError("Invalid count_type. Should be 'add', 'remove', or 'total'.")

    df.to_csv('./data/fridge_inventory.csv',index = False)
    


def fridge_reset():
    f = pd.DataFrame({"item":[],"quantity":[]})
    f.to_csv('./data/fridge_inventory.csv', index = False)

In [9]:
fridge_reset()
pd.read_csv('./data/fridge_inventory.csv')

,item,quantity


In [10]:
def fridge_mod(message, tools = tools):
    messages = [
        {
            "role": "user",
            "content": message
        }
    ]
    run_conversation(messages,tools)
    return pd.read_csv('./data/fridge_inventory.csv')

In [11]:
fridge_mod("How many bananas are in the fridge?")

BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[1].function'.", 'type': 'invalid_request_error', 'param': 'tools[1].function', 'code': 'missing_required_parameter'}}

In [13]:
messages = [
    {
        "role": "user",
        "content": "I put a banana in the fridge"
    }
]

In [14]:
run_conversation(messages,tools)
fridge = pd.read_csv('./data/fridge_inventory.csv')
fridge

BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[1].function'.", 'type': 'invalid_request_error', 'param': 'tools[1].function', 'code': 'missing_required_parameter'}}